In [ ]:
import numpy as np
import scipy

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('jf')

In [ ]:
from jftools import fedvr

In [ ]:
# 5 points (element boundaries) gives 4 elements
# very low order to have only a few basis functions for plot
# g = fedvr_grid(4,np.linspace(0,8,5))
g = fedvr.fedvr_grid(4,np.array([0,2,3.5,4.5,6,8]))
xnew = np.linspace(g.x[0],g.x[-1],500)
fvals = g.get_basis_function_values(xnew)
plt.plot(g.x,0*g.x,'ro',ms=10,mew=2.5,zorder=5,label='FEDVR points')
for x in [r.x[0] for r in g.regs]+[g.regs[-1].x[-1]]:
    plt.axvline(x,ls='--',color='0.4',lw=1,label='FE boundaries' if x==g.regs[0].x[0] else None)
plt.plot(xnew,fvals.T)
plt.margins(0.03)
plt.legend()
plt.tight_layout(pad=0)

In [ ]:
from ipywidgets import interact
import scipy.sparse.linalg

g = fedvr.fedvr_grid(11,np.linspace(-80,80,41))
print("#Grid points:",len(g.x))
M = 1.
sigma = 8.
k0 = 1.
ts, dt = np.linspace(0,300,301,retstep=True)
f0 = lambda x: np.exp(-x**2/(2*sigma**2) + 1j*k0*x)
H = -g.dx2/(2*M)
psis = np.zeros([len(ts),len(g.x)],dtype=complex)
psis[0] = g.project_function(f0)
U = scipy.sparse.linalg.expm(-1j*dt*H.tocsc())
for ii in range(1,len(ts)):
    psis[ii] = U.dot(psis[ii-1])

xnew = np.linspace(g.x[0],g.x[-1],500)
psiplots = g.evaluate_basis(psis,xnew)
@interact(ii=(0,len(ts)-1))
def doplot(ii=0):
    plt.plot(xnew,abs(psiplots[ii])**2)
    #plt.plot(g.x,abs(psis[ii])**2/g.wt,'o--')
    plt.ylim(0,1)

In [ ]:
sigma = 0.8
fdfs = [(lambda x: np.exp(-x**2/(2*sigma**2)), lambda x: np.exp(-x**2/(2*sigma**2)) * -x/sigma**2, lambda x: np.exp(-x**2/(2*sigma**2)) * (x**2-sigma**2)/sigma**4),
        (np.sin, np.cos, lambda x: -np.sin(x)),
        (lambda x: np.sin(np.pi*x/4)**2, lambda x: np.pi/4*np.sin(np.pi*x/2), lambda x: np.pi**2/8*np.cos(np.pi*x/2)),
        (lambda x: np.pi/4*np.sin(np.pi*x/2), lambda x: np.pi**2/8*np.cos(np.pi*x/2), lambda x: -np.pi**3/16*np.sin(np.pi*x/2)),
        (lambda x: np.sin(x)**2, lambda x: np.sin(2*x), lambda x: 2*np.cos(2*x)),
        (lambda x: np.sin(12*x), lambda x: 12*np.cos(12*x), lambda x: -144*np.sin(12*x))
       ]
g = fedvr.fedvr_grid(11,np.linspace(-4,4,5))
xnew = np.linspace(g.x[0],g.x[-1],1000)

fig, axs = plt.subplots(1,len(fdfs),figsize=(7.5*len(fdfs),6.5))
for (f,df,d2f),ax in zip(fdfs,axs):
    cn = g.project_function(f)
    y  = g.evaluate_basis(cn,xnew)
    dcn = g.dx.dot(cn)
    dy  = g.evaluate_basis(dcn,xnew)
    dcn2 = g.dx2.dot(cn)
    dcn2a = g.dx.dot(dcn)
    dy2  = g.evaluate_basis(dcn2,xnew)
    dy2a = g.evaluate_basis(dcn2a,xnew)
    next(ax._get_lines.prop_cycler)
    ax.plot(xnew,y,label=r'$f(x)$')
    ax.plot(xnew,f(xnew),'k--')
    ax.plot(xnew,dy,label=r"$f'(x)$")
    ax.plot(xnew,df(xnew),'k--')
    ax.plot(xnew,dy2,label=r"$f''(x)$")
    ax.plot(xnew,d2f(xnew),'k--')
    ax.margins(0.03)
    ax.legend(fontsize=18)
fig.tight_layout()

In [ ]:
dx = g.dx.toarray()
dxdx = dx @ dx
dx2 = g.dx2.toarray()
print(np.linalg.norm(dx+dx.T))
print(np.linalg.norm(dxdx-dxdx.T))
print(np.linalg.norm(dx2-dx2.T))
plt.plot(np.linalg.eigvalsh(-0.5*dx2))
plt.plot(np.linalg.eigvalsh(-0.5*dxdx))
plt.plot(np.arange(dx.shape[0])**2*np.pi**2/(2*8**2))
plt.ylim(0,100)
#plt.xlim(0,10); plt.ylim(0,10)

In [ ]:
f, axs = plt.subplots(1,4,figsize=(23,4))
for ax, arr in zip(axs,[dx,dx2,dxdx,dx2-dxdx]):
    arr = np.sign(arr)*np.sqrt(abs(arr))
    vmax = abs(arr).max()
    im = ax.imshow(arr,interpolation='none',cmap='coolwarm',vmin=-vmax,vmax=vmax)
    plt.colorbar(im,ax=ax)

In [ ]:
fdfs = [(lambda x: np.exp(-x**2/(2*0.5**2)), lambda x: np.exp(-x**2/(2*0.5**2)) * -x/0.5**2),
        (np.sin, np.cos),
        (lambda x: np.sin(x)**2, lambda x: np.sin(2*x)),
        (lambda x: np.sin(12*x), lambda x: 12*np.cos(12*x))
       ]
g = fedvr.fedvr_grid(11,np.linspace(-4,4,5))
xnew = np.linspace(g.x[0],g.x[-1],1000)

fig, axs = plt.subplots(1,len(fdfs),figsize=(7*len(fdfs),5.5))
for (f,df),ax in zip(fdfs,axs):
    cn = g.project_function(f)
    y  = g.evaluate_basis(cn,xnew)
    dcn = g.dx.dot(cn)
    dy  = g.evaluate_basis(dcn,xnew)
    next(ax._get_lines.prop_cycler)
    ax.plot(xnew,y,label=r'$f(x)$')
    ax.plot(xnew,f(xnew),'k--')
    ax.plot(xnew,dy,label=r"$f'(x)$")
    ax.plot(xnew,df(xnew),'k--')
    ax.margins(0.03)
    ax.legend(fontsize=18)
fig.tight_layout()

In [ ]:
f = lambda x: np.exp(-x**2/(2*0.5**2))
nfuns = [5,8,11,15]
fig, axs = plt.subplots(1,len(nfuns),figsize=(7*len(nfuns),5.5),sharey=True)
for nfun, ax in zip(nfuns,axs):
    g = fedvr.fedvr_grid(nfun,np.linspace(-4,4,5))
    xnew = np.linspace(g.x[0],g.x[-1],1000)
    y = f(xnew)
    cn = g.project_function(f)
    ynew = g.evaluate_basis(cn,xnew)
    ax.plot(xnew,y,label=r'$f(x)$',lw=3)
    ax.plot(g.x,cn/np.sqrt(g.wt),'o--',lw=1,ms=6,label=r'$f(x_n) = c_n/\sqrt{w_n}$',zorder=4)
    ax.plot(xnew,ynew,'--',label=r'$\tilde f(x) = \sum_n c_n b_n(x)$')
    ax.margins(0.02)
    ax.legend()
    ax.set_title(r"$N_{fun} = %d$, $\|\tilde f - f\|/\|f\| = %.3e$"%(nfun,np.trapz(abs(y-ynew),xnew)/np.trapz(y,xnew)),verticalalignment='bottom')
    print(np.trapz(y,xnew)-np.sum(cn*np.sqrt(g.wt)))
fig.tight_layout(pad=0.5)